# Problem2: Optimal Bayesian Estimation

# 0- First Some Data Pre-Processing

- At this moment you should have "train_PCA.pkl" and "test_PCA.pkl" files in your working directory

In [1]:
import pickle
import numpy as np
import math


train_data=pickle.load(open( "train_PCA.pkl", "rb" ),encoding ='latin1')
test_data=pickle.load(open( "test_PCA.pkl", "rb" ),encoding ='latin1')


In [2]:
categories = ['alt.atheism',
              'talk.religion.misc',
              'comp.graphics',
              'sci.space']

num_classes = len(categories)

In [3]:
val_ratio  = 0.1 
e=math.e
L = [e**-25, e**-20, e**-15, e**-10, e**-5,0,1,2,3, e**5, e**10] # range of lambda

In [4]:
#Split Train data into Train and Validation (Ratio Train : Val = 4:1)

train_num = int(train_data['data'].shape[0]*(1.0-val_ratio)) 
val_num = -1*int(train_data['data'].shape[0]*val_ratio)

In [5]:
#Train data and Train Target
#Validation data and Validation Target


train_feature = train_data['data'][:train_num]
train_target = train_data['target'][:train_num]

val_feature = train_data['data'][val_num:]
val_target = train_data['target'][val_num:]

test_feature = test_data['data']
test_target = test_data['target']
test_num = test_data['data'].shape[0] 

In [6]:
len(test_target)

1353

In [7]:
# shape of data set

print(train_target.shape)
print(val_target.shape)

(1830,)
(203,)


# 1- Find Mean and Prior for Estimating Multivariate Normal Dist. for Classes

In [8]:
# Find the mean and prior (which is just the frequency)
mu = np.zeros([train_feature.shape[1],len(categories)], dtype=float)
Pi = np.zeros(len(categories),dtype=float)

for i in range(len(train_target)):
    Pi[train_target[i]] += 1
    mu[:,train_target[i]] += train_feature[i]


for j in range(4):
    mu[:,j] = mu[:,j]/Pi[j]

Pi = Pi/len(train_feature)  


In [9]:
print (np.shape(mu[:,0]))
print (np.shape(test_feature[0]))

(1500,)
(1500,)


# 2- Build Models For Several Cases

For each case consider the followings:


- First, compute the covariance matrix (name it cov)

- Build the Multivariate Normal dist. based on the computed mean and cov.

- Then build the Model

- Report the Confusion Matrix, ROC and plot the data scatter and the decision boundaries in 2D (so based on top-2 PCA features). For the Confusion matrix and ROC, use Scilearn built in functions.

 *Case 0: multivariate normals with shared spherical variances:*
 
- In this case, first compute the cov to be a diagonal matrix with the same element on diag which is the average of the feasure variances, i.e. sum(diag(cov(x)))/2*eye(2), x is the data point matrix.

In [10]:
import scipy.stats as stats
import math
from sklearn.metrics import confusion_matrix

In [11]:
cov = sum(np.diag(np.cov(train_feature.T)))/1500*np.eye(1500)

In [12]:
def pred_shared(cov):    
    sigma=sum(np.diag(np.cov(train_feature.T)))/1500

    w=np.zeros([len(categories),train_feature.shape[1]],dtype=float)
    w0=np.zeros(len(categories),dtype=float)
    g=np.zeros([len(categories),train_feature.shape[1]],dtype=float)
    for i in range(4):
        mu_=mu[:,i]
        w[i]=np.divide(mu_,sigma)
        w0[i]=1.0/(2*sigma)* np.matmul(mu_, mu_)+ np.log(Pi[i])

    prediction = []

    for i in range(len(test_feature)):
        g=np.zeros(4,dtype=float)
        for c in range(4):
            g[c]=np.matmul(w[c],test_feature[i])+w0[c]
        prediction.append(np.argmax(g))
    return prediction

In [13]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
import sklearn.preprocessing
def draw_plt(num):
    label_binarizer = sklearn.preprocessing.LabelBinarizer()
    label_binarizer.fit(range(4))

    fpr = dict()
    tpr = dict()
    roc_auc = dict()


    test_vectorized = label_binarizer.transform(test_target)
    y =  label_binarizer.transform(prediction)

    for i in range(4):
        fpr[i], tpr[i], _ = roc_curve(test_vectorized[:,i],y[:,i]) # i column
        roc_auc[i] = auc(fpr[i], tpr[i])
    print ("roc accuracy:")
    print (roc_auc)
    
    lw = 2
    for i in range(4):
        plt.figure()
        plt.plot(fpr[i], tpr[i], color='darkorange',
             lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[i])
        plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver operating characteristic example_group'+str(i))
        plt.legend(loc="lower right")
        filename="Q6_case"+num+"_ROC_"+str(i)+".png"
        plt.savefig(filename)
        plt.show()

In [14]:
from sklearn.decomposition import PCA
pca=PCA(n_components=2)
pca.fit(train_feature)
train_pca=pca.transform(train_feature)
train_class_pca=dict()
for i in range(4):
    train_class_pca[i]=pca.transform(train_class[i])
mu_pca = np.zeros([2,len(categories)], dtype=float)
for c in range(4):
    mu_pca[:,c]=np.mean(train_class_pca[c],axis=0).transpose()
cov_pca=np.cov(train_pca.transpose())
sigma_pca=sum(np.diagonal(cov_pca))/2
w_p=np.zeros([len(categories),2],dtype=float)
w0_p=np.zeros(len(categories),dtype=float)
for i in range(4):
    mu_p=mu_pca[:,i]
    w_p[i]=np.divide(mu_p,sigma_pca)
    w0_p[i]=1.0/(2*sigma_pca)* np.matmul(mu_p, mu_p)+ np.log(Pi[i])


NameError: name 'train_class' is not defined

In [ ]:
prediction = pred_shared(cov)
print ("confusion matrix:")
print (confusion_matrix(prediction, test_target))
draw_plt('0')

*Case 1: multivariate normals with shared axis parellel variances:*

- In this case, first compute the cov to be the diagonal matrix with the variance of each feasure on the diag, i.e. diag(diag(cov(x))).

In [ ]:
cov = np.diag(np.diag(np.cov(train_feature.T)))

In [ ]:
prediction = []
cov_inv = np.linalg.inv(cov)
for i in range(len(test_feature)):
    mx = -1
    idx = -1
    for j in range(len(categories)):
        y = test_feature[i] - mu[:,j]
        y = np.dot(np.dot(y.T,cov_inv),y)
        y = math.exp(-0.5*y)*Pi[j]
        if y > mx:
            mx = y
            idx = j
    prediction.append(idx)

In [ ]:
confusion_matrix(prediction, test_target)

*Case 2: multivariate normals with shared arbitrary variances:*

- In this case, first compute cov to be the general covariance matrix of feasures, i.e. cov(x).

In [ ]:
cov = np.cov(train_feature.T)

In [ ]:
prediction = []
cov_inv = np.linalg.inv(cov)
for i in range(len(test_feature)):
    min = 10000000
    idx = -1
    for j in range(4):
        y = test_feature[i] - mu[:,j]
        y = np.dot(np.dot(y.T,cov_inv),y)
#         y = math.exp(-0.5*y)
        if y < min:
            min = y
            idx = j
    prediction.append(idx)

In [ ]:
confusion_matrix(prediction, test_target)

*Case 3: multivariate normals with shared spherical variances:*

- In this case, first compute a cov for each class.

In [ ]:
cov = np.zeros([train_feature.shape[1],train_feature.shape[1],len(categories)])

In [ ]:
for i in range(len(categories)):
    train_special_feature = []
    for tar in train_target:
        if tar == i :
            train_special_feature.append(train_feature[:,i])
    cov
            


Case 4: Same as case 3, but with reject option. Threshold 0.3